In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io
import itertools as it
import scipy.special as psi
plt.style.use('classic')
import seaborn as sns
import pandas as pd
import math as mt
import time
import sys

sys.path.insert(1, '/rds/general/user/lr4617/home/4th_Year_Project/Final_Year_Project/')

from scipy.io import loadmat
from scipy import stats
from numpy.random import seed
from numpy.random import rand
from scipy.integrate import quad
from scipy.io import savemat
from tempfile import TemporaryFile
from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.decomposition import KernelPCA
from mpl_toolkits import mplot3d
from mPE_fn import mPE_
from scipy.spatial import distance
from scipy.stats import entropy
from mPE_ultis import integrand, ubble, array_list, permutation
from util import rolling_mean, probability, probability_v2, get_mPE_matrix
from sklearn.cluster import KMeans

In [2]:
rats = np.load('/rds/general/user/lr4617/home/4th_Year_Project/Final_Report/control_analysis/rats_sampling_10_window_150.npy')
print(rats.shape)
lengths = np.load('/rds/general/user/lr4617/home/4th_Year_Project/Final_Report/control_analysis/lengths.npy')

to_be_removed = '.ipynb_checkpoints'

root_dir = '/rds/general/user/lr4617/home/4th_Year_Project/CAPTURE_rat_multidimensional/raw_data/'
normal_trajs = os.listdir(root_dir + 'normal')
drug_trajs_1 = os.listdir(root_dir + 'caffeine')
drug_trajs_2 = os.listdir(root_dir + 'amphetamine')

if to_be_removed in normal_trajs:
    idx = normal_trajs.index(to_be_removed)
    normal_trajs.pop(idx)
    
if to_be_removed in drug_trajs_1:
    idx = drug_trajs_1drug_trajs_1.index(to_be_removed)
    drug_trajs_1.pop(idx)

if to_be_removed in drug_trajs_2:
    idx = drug_trajs_2.index(to_be_removed)
    drug_trajs_2.pop(idx)

names = normal_trajs + drug_trajs_1 + drug_trajs_2

(6480000, 60)


In [6]:
###################################################################
###################### DMI: TIME-DEPENDENT ########################
###################################################################

DMIs = np.zeros((20,50,3))

modes = ['normal', 'caffeine', 'amphetamine']

block_lengths = [sum(lengths[i*4:(i*4)+4]) for i in range(len(modes))]
sample_size = 30000
dims = np.arange(60)
n_PC = 3
min_lengths = [334800, 334800, 334800]

for i in range(len(block_lengths)):

    print(modes[i])
    
    if i == 0:
        signal_mode = rats[0:block_lengths[0], :]
        print(signal_mode.shape)
    else:
        signal_mode = rats[block_lengths[i-1]:block_lengths[i-1]+block_lengths[i], :]

    for ii in range(4):
    
        if ii == 0 :
            signal = signal_mode[0:lengths[(i*4)], :]
        else:
            signal = signal_mode[ lengths[(i*4) + ii -1]: lengths[(i*4) + ii-1] + lengths[(i*4) + ii], :]
            
            if i == 2 and ii == 1:
                prev = signal_mode[lengths[(i*4) + ii - 2]:lengths[(i*4) + ii-2] + lengths[(i*4) + ii -1], :]
                signal[signal.shape[0]:min_lengths[i], :] = prev[signal.shape[0]:min_lengths[i], :]
        
        pca = PCA(n_components=n_PC)
        reduced_traj = pca.fit_transform(signal)
        
        signal = signal[0:min_lengths[i], :]
        reduced_traj = reduced_traj[0:min_lengths[i], :]
        
        if ii == 0:
            rats_ = signal
            rats_red = reduced_traj
        else:
            rats_ = np.concatenate((rats_, signal), axis=0)
            rats_red = np.concatenate((rats_red, reduced_traj), axis=0)
        
    for iii in range(0,4*min_lengths[i],4*sample_size):
    
        # red_signal = rats_red[iii:iii+4*sample_size, :]
        # [rH, _] = mPE_(red_signal, 3)

        for dim in range(int(len(dims)/3)):
            
            print('dimension = ', dim)
            
            # retrieve marker signal (x,y,z)
            marker_signal = rats_[iii:iii+4*sample_size, dim:dim+3]
            
            print(marker_signal.shape, red_signal.shape)

            # calculate marker signal mPE
            [mH, _] = mPE_(marker_signal, 3)

            # create joint signal for Joint Dynamical Entropy
            joint_signal = np.concatenate((red_signal, marker_signal), axis=1)
            print(joint_signal.shape)
            
            # caluclate joint entropy using joint signal
            [JH, _] = mPE_(joint_signal, 3)

            # calculate dynamical mutual information
            DMI = JH - mH
            
            print(DMI, JH, mH)
            
            # fill array
            DMIs[dim, int(iii/(4*sample_size)), i] = DMI
            
            

normal
(3942000, 60)
dimension =  0
(120000, 3) (120000, 3)
(120000, 6)
[[3.44025107]] [[8.6294359]] [[5.18918484]]
dimension =  1
(120000, 3) (120000, 3)
(120000, 6)
[[3.43626844]] [[8.58166723]] [[5.1453988]]
dimension =  2
(120000, 3) (120000, 3)
(120000, 6)
[[3.42337035]] [[8.50675965]] [[5.08338931]]
dimension =  3
(120000, 3) (120000, 3)
(120000, 6)
[[3.35985276]] [[8.50031424]] [[5.14046147]]
dimension =  4
(120000, 3) (120000, 3)
(120000, 6)
[[3.35976172]] [[8.53620556]] [[5.17644384]]
dimension =  5
(120000, 3) (120000, 3)
(120000, 6)
[[3.36139983]] [[8.57424383]] [[5.212844]]
dimension =  6
(120000, 3) (120000, 3)
(120000, 6)
[[3.37365575]] [[8.56400599]] [[5.19035023]]
dimension =  7
(120000, 3) (120000, 3)
(120000, 6)
[[3.30876929]] [[8.41606495]] [[5.10729566]]
dimension =  8
(120000, 3) (120000, 3)
(120000, 6)
[[3.29404366]] [[8.17746192]] [[4.88341825]]
dimension =  9
(120000, 3) (120000, 3)
(120000, 6)
[[3.13746154]] [[7.9691858]] [[4.83172426]]
dimension =  10
(120000,

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 120000 and the array at index 1 has size 19200

In [ ]:
###################################################################
###################### DMI: TIME-INDEPENDENT ######################
###################################################################
DMIs = np.zeros((len(lengths), 20))

for which_traj, length in enumerate(lengths):

    print('TRAJECTORY ', str(which_traj))
    
    # retrieve trajectory from trajectories
    if which_traj == 0:
        idx = 0
    else:
        idx += lengths[which_traj-1]

    traj = rats[idx:idx+length, :]

    print('########################## APPLY PCA ##########################')
    
    # apply PCA to high-d signal to reduce it to "n_PC" dims
    pca = PCA(n_components=n_PC)
    reduced_traj = pca.fit_transform(traj)
    
    # calculate marker signal mPE
    [HH, _]=mPE_(reduced_traj, 3)
    
    for dim in range(0,traj.shape[1],3):
        
        # retrieve marker signal (x,y,z)
        marker_signal = traj[:, dim:dim+3]
        
        # calculate marker signal mPE
        [mH, _] = mPE_(marker_signal, 3)
        
        # create joint signal for Joint Dynamical Entropy
        joint_signal = np.concatenate((reduced_traj, marker_signal), axis=1)
        
        # caluclate joint entropy using joint signal
        [JH, _] = mPE_(joint_signal, 3)
        
        # calculate dynamical mutual information
        DMI = HH + mH - JH
        
        DMIs[which_traj, int(dim/3)] = DMI[0]
        
        print(DMI[0])

path_out = '/rds/general/user/lr4617/home/4th_Year_Project/Final_Report/'
name_out = 'marker_DMI.npy'
np.save(path_out + name_out, DMIs)